In [1]:
## copy iz started_doing_something_MV_model_ALL_spots_with_CB_cancellation_requested_reports notbuka

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import math
import importlib
from dateutil.relativedelta import relativedelta
import get_started_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

#### Helpers ####
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import db_utils
from helpers import settings

bucket = S3BucketUtils()
################

import calculate_all_vars_for_churn_prediction
import churn_risk_calculation_test
import churn_risk_calculation
import model_testing

import re

import models_to_be_fit

In [3]:
settings.S3_ANALYTICS_BUCKET

'spothopper-analytics-kiza'

In [4]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
hs_list_filename = parameters['hs_list_filename']
hs_list_path = 'churn_analysis/data/'+date_dir+'/'+hs_list_filename

prediction_months = parameters['prediction_months']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'CAN_CANCEL_spots_wo_CB_cancellation_requested'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [5]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [6]:
date_dir

'2023_01_01'

In [7]:
website_sessions_wo_contactless = pd.read_csv('../issue_3788_mixpanel_tv/data/' + date_dir + \
                                              '/GA_data/sessions_new.csv')

website_sessions_wo_contactless = website_sessions_wo_contactless.groupby(['spot_id', 'event_time'])\
.sessions.sum().reset_index()

website_sessions_wo_contactless['event_time'] = website_sessions_wo_contactless['event_time']\
.apply(lambda x: x[0:7])

website_sessions_wo_contactless = website_sessions_wo_contactless.groupby(['spot_id', 'event_time'])\
.sessions.sum().reset_index()

website_sessions_wo_contactless = website_sessions_wo_contactless.groupby(['spot_id'])\
.sessions.mean().reset_index()

In [8]:
##### Model 41: all variables - prepare data for the model ####
cols_to_use_ = read_a_combination_of_variables.\
main(model_number=41, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use_)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

# data_all_spots = \
# calculate_all_vars_for_churn_prediction.\
# get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis,\
#                               hs_filename=hs_list_filename,\
#                               spots_set=spots_set,\
#                               with_wo_CB=with_wo_CB,\
#                               event_date=event_date, columns=cols,\
#                               data_dir=churn_based_on_behaviour_dir)

#### get data for the model ###
(data_, base_df_, df_timeline_all_vars_) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [9]:
num_of_p_values = 1
num_of_prediction_months = len(prediction_months)

In [10]:
df_res = pd.DataFrame()
models_to_run = [36, 37, 38, 39, 40]
cut_offs = [500, 700, 1000, 1200, 1500, 1700, 2000]
for model_number in models_to_run:
    for ct in cut_offs:
        for e in ('high_end_', 'low_end_'):
            if e=='high_end_':
                spots_ = website_sessions_wo_contactless[website_sessions_wo_contactless['sessions']>=ct]['spot_id'].unique()
            else:
                spots_ = website_sessions_wo_contactless[website_sessions_wo_contactless['sessions']<ct]['spot_id'].unique()
            
            data = data_[data_['spot_id'].isin(spots_)].copy()
            base_df = base_df_[base_df_['spot_id'].isin(spots_)].copy()
            df_timeline_all_vars = df_timeline_all_vars_[df_timeline_all_vars_['spot_id'].isin(spots_)].copy()
            
            cols_to_use = read_a_combination_of_variables.\
            main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')

            model_name = \
            read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                            dir_name='combinations_of_variables_that_are_not_dependent/')
            print("# Model "+ str(model_number) + ": " + model_name)
            
            print('had_'+model_name)

            #### get behavioural variables ####
            (variables_to_use_for_the_model, did_something_last_month_vars, \
             did_something_before_and_didnt_last_month_vars) = \
            get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, \
                                                       variables_to_use_for_the_model=cols_to_use)
            cols = variables_to_use_for_the_model + \
            did_something_before_and_didnt_last_month_vars + did_something_last_month_vars
            
            variable_to_check = [c for c in cols if c.startswith('had_'+model_name) and \
                   re.search('before_and_didnt', c) == None][0]
            
            print(variable_to_check)

            #### data for the model ###
            df_timeline = df_timeline_all_vars.copy()
            vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
            [x for x in cols if x not in base_cols]+\
            [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
            df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

            ### variables to skip ###
            df_timeline.isnull().sum().sum() #OK
            skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
            skip_vars

            ctv = fit_tv_cox_models.fit_the_models_and_print_summaries_kiza(df_timeline=df_timeline, base_df=base_df, \
                                                                 date_of_analysis=date_of_analysis, model_type=model_type, \
                                                                 variables_to_use_for_the_model=cols_to_use,\
                                                                 coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                                 coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                                 additional_higher_p_limit=additional_higher_p_limit, \
                                                                 additional_lower_p_limit=additional_lower_p_limit,\
                                                                 skip_vars=skip_vars, penalizer=penalizer)
            
            df_res = pd.concat([df_res, pd.DataFrame([[model_number, e + str(ct), 
                                                       model_name, variable_to_check, ctv, data.spot_id.nunique()]])])

# Model 36: properly_used_reservations_inquiries
had_properly_used_reservations_inquiries
had_properly_used_reservations_inquiries_last_2_months


### sorted by variable name

## without variables with p >= 0.2

### sorted by variable name

# Model 36: properly_used_reservations_inquiries
had_properly_used_reservations_inquiries
had_properly_used_reservations_inquiries_last_2_months


...

### sorted by variable name

# Model 40: properly_used_online_orders_inquiries
had_properly_used_online_orders_inquiries
had_properly_used_online_orders_inquiries_last_2_months


### sorted by variable name

## without variables with p >= 0.2

### sorted by variable name

In [11]:
df_res.columns = ['model_number', 'category', 'model_name', 'variable_name', 'models', 'spots_num']

In [12]:
df_res.columns.tolist()

['model_number',
 'category',
 'model_name',
 'variable_name',
 'models',
 'spots_num']

In [13]:
def extract_coefficients(variable_name, model):
    model_summary = model.summary
    if variable_name in model_summary.index:
        return(model_summary[model_summary.index==variable_name]['exp(coef)'].iloc[0])
    else:
        return(np.NaN)

def extract_p_values(variable_name, model):
    model_summary = model.summary
    if variable_name in model_summary.index:
        return(model_summary[model_summary.index==variable_name]['p'].iloc[0])
    else:
        return(np.NaN)

In [14]:
df_res['exp(coef)'] = df_res.apply(lambda x: extract_coefficients(variable_name=x['variable_name'], 
                                                                  model=x['models'][0]), axis=1)
df_res['p'] = df_res.apply(lambda x: extract_p_values(variable_name=x['variable_name'], 
                                                                  model=x['models'][0]), axis=1)

df_res['exp(coef).2'] = df_res.apply(lambda x: extract_coefficients(variable_name=x['variable_name'], 
                                                                  model=x['models'][1]), axis=1)
df_res['p.2'] = df_res.apply(lambda x: extract_p_values(variable_name=x['variable_name'], 
                                                                  model=x['models'][1]), axis=1)

In [15]:
def check_model(summary):
    vars_ = summary.index
    vars_ = [x[4:] if x.startswith('had_') else x for x in vars_]
    vars_ = [x[:-14] if x.endswith('_months') else x for x in vars_]
    vars_ = [x[:-17] if x.endswith('_before_and_didnt') else x for x in vars_]
    vars_ = [x[:-32] if x.endswith('_before_and_didnt_last_4_months\t') else x for x in vars_]
    vars_ = [i for i in vars_ if not(i.startswith('metro_area_') or i.startswith('spot_category_'))]
    vars_ = [x[:-15] if x.endswith('_last_4_months\t') else x for x in vars_]
    vars_ = sorted(pd.Series(vars_).unique())
    return(len(vars_))

In [16]:
[len(sorted(read_a_combination_of_variables.\
main(model_number=n, dir_name='combinations_of_variables_that_are_not_dependent/'))) for n 
 in [36, 37, 38, 39, 40]] 

[68, 68, 68, 68, 68]

In [17]:
df_res[df_res['model_number']==36][['model_number', 'category', 'model_name', 'variable_name']]

,model_number,category,model_name,variable_name
0,36,high_end_500,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,low_end_500,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,high_end_700,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,low_end_700,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,high_end_1000,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,low_end_1000,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,high_end_1200,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,low_end_1200,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,high_end_1500,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...
0,36,low_end_1500,properly_used_reservations_inquiries,had_properly_used_reservations_inquiries_last_...


In [18]:
df_res[df_res['model_number']==36]['models'].map(lambda x: check_model(x[0].summary))

0    68
0    66
0    68
0    66
0    68
0    67
0    68
0    67
0    68
0    67
0    68
0    67
0    67
0    68
Name: models, dtype: int64

In [19]:
# df_res[['model_number', 'category', 'model_name', 'variable_name', 'spots_num', 
#         'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

In [20]:
df_res_export = df_res[['model_number', 'category', 'model_name', 'variable_name', 'spots_num', 
        'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

In [21]:
df_res_export[df_res_export['model_number']==39]

,model_number,category,model_name,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,39,high_end_500,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,2353,1.037196,0.874724,NaN,NaN
0,39,low_end_500,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,219,0.253921,0.701943,NaN,NaN
0,39,high_end_700,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,2246,1.106328,0.667654,NaN,NaN
0,39,low_end_700,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,326,0.159983,0.323348,NaN,NaN
0,39,high_end_1000,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,2023,1.147338,0.571590,NaN,NaN
0,39,low_end_1000,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,549,0.144368,0.096493,0.155964,0.097806
0,39,high_end_1200,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,1874,1.229224,0.409980,NaN,NaN
0,39,low_end_1200,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,698,0.190610,0.090525,0.165350,0.055223
0,39,high_end_1500,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,1665,1.257852,0.377764,NaN,NaN
0,39,low_end_1500,properly_used_job_listings_inquiries,had_properly_used_job_listings_inquiries_last_...,907,0.241478,0.068725,0.207087,0.035242


In [22]:
df_res_export[df_res_export['model_number']==40][::-1]

,model_number,category,model_name,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,40,low_end_2000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1248,0.811496,0.562669,NaN,NaN
0,40,high_end_2000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1324,1.254032,0.542681,NaN,NaN
0,40,low_end_1700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1047,0.867642,0.724516,NaN,NaN
0,40,high_end_1700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1525,1.120845,0.737009,NaN,NaN
0,40,low_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,907,0.872238,0.754278,NaN,NaN
0,40,high_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1665,1.146658,0.672926,NaN,NaN
0,40,low_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,698,0.659408,0.399738,NaN,NaN
0,40,high_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1874,1.164636,0.617500,NaN,NaN
0,40,low_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,549,0.717318,0.571293,NaN,NaN
0,40,high_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2023,1.096789,0.749796,NaN,NaN


In [23]:
df_res[df_res_export['model_number']==40]['models'].iloc[0][0].summary[df_res[df_res_export['model_number']==40]['models'].iloc[0][0].summary.index==\
'had_properly_used_online_orders_inquiries_last_2_months']

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
had_properly_used_online_orders_inquiries_last_2_months,-0.006583,0.993438,0.263991,-0.523997,0.51083,0.592149,1.666674,0.0,-0.024937,0.980105,0.028991


In [24]:
df_res_export[df_res_export['model_number']==36][::-1]\
.to_csv('properly_used_reservations_set_CAN_CANCEL_01_01_2023.csv', index=False)

In [25]:
df_res_export[df_res_export['model_number']==37][::-1]\
.to_csv('properly_used_catering_set_CAN_CANCEL_01_01_2023.csv', index=False)

In [26]:
df_res_export[df_res_export['model_number']==38][::-1]\
.to_csv('properly_used_private_parties_set_CAN_CANCEL_01_01_2023.csv', index=False)

In [27]:
df_res_export[df_res_export['model_number']==39][::-1]\
.to_csv('properly_used_job_listings_set_CAN_CANCEL_01_01_2023.csv', index=False)

In [28]:
df_res_export[df_res_export['model_number']==40][::-1]\
.to_csv('properly_used_online_orders_set_CAN_CANCEL_01_01_2023.csv', index=False)

In [29]:
df_res[df_res['model_number']==40][['model_number', 'category', 
                                    'model_name', 'variable_name', 
                                    'spots_num', 'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

,model_number,category,model_name,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,40,high_end_500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2353,0.993438,0.980105,NaN,NaN
0,40,low_end_500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,219,0.181145,0.189404,0.119482,0.013914
0,40,high_end_700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2246,0.978642,0.936765,NaN,NaN
0,40,low_end_700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,326,0.539847,0.451112,NaN,NaN
0,40,high_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2023,1.096789,0.749796,NaN,NaN
0,40,low_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,549,0.717318,0.571293,NaN,NaN
0,40,high_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1874,1.164636,0.617500,NaN,NaN
0,40,low_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,698,0.659408,0.399738,NaN,NaN
0,40,high_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1665,1.146658,0.672926,NaN,NaN
0,40,low_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,907,0.872238,0.754278,NaN,NaN


In [30]:
df_res[(df_res['model_number']==40) & (df_res['category'].isin(['low_end_500']))]\
[['model_number', 'category', 'model_name', 
  'variable_name', 'spots_num', 'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

,model_number,category,model_name,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,40,low_end_500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,219,0.181145,0.189404,0.119482,0.013914


In [31]:
df_res[(df_res['model_number']==40) & (df_res['category'].isin(['low_end_500']))]['models'].iloc[0][0]\
.summary.index

Index(['Annual Commitment with 2 semi-annual payments', 'Annual Prepay',
       'Fb.page.likes.analysis.avg.log2', 'Monthly',
       'New.email.subscriber.signups.last.month.div10',
       'New.email.subscribers.allinclusive.last.month.div10',
       'Opted.out.of.facebook', 'SpotHopper_site_not_live_yet',
       'Website.Views.last.month.total.log2',
       'had_added_events_manually_edited_events_before_and_didnt_last_4_months',
       ...
       'spot_category_Sports Bar', 'spot_category_Sushi',
       'spot_category_other', 'their_own_website_dontKnow',
       'their_own_website_no', 'year_become_customer_2018.0',
       'year_become_customer_2019.0', 'year_become_customer_2020.0',
       'year_become_customer_2021.0', 'year_become_customer_2022.0'],
      dtype='object', name='covariate', length=149)

In [32]:
def compare_models_vars(model_0, model_1):
    
    vars_ = model_0.summary.index
    vars_ = [x[:-1] if x.endswith('\t') else x for x in vars_]
    vars_ = [x[4:] if x.startswith('had_') else x for x in vars_]
    vars_ = [x[:-14] if x.endswith('_months') else x for x in vars_]
    vars_ = [x[:-17] if x.endswith('_before_and_didnt') else x for x in vars_]
    vars_ = [x[:-32] if x.endswith('_before_and_didnt_last_4_months') else x for x in vars_]
    # vars_ = [i for i in vars_ if not(i.startswith('metro_area_') or i.startswith('spot_category_'))]
    # vars_ = [x[:-15] if x.endswith('_last_4_months\t') else x for x in vars_]
    vars_ = sorted(pd.Series(vars_).unique())
    
    vars_1 = model_1.summary.index
    vars_1 = [x[:-1] if x.endswith('\t') else x for x in vars_1]
    vars_1 = [x[4:] if x.startswith('had_') else x for x in vars_1]
    vars_1 = [x[:-14] if x.endswith('_months') else x for x in vars_1]
    vars_1 = [x[:-17] if x.endswith('_before_and_didnt') else x for x in vars_1]
    vars_1 = [x[:-32] if x.endswith('_before_and_didnt_last_4_months') else x for x in vars_1]
    # vars_1 = [i for i in vars_1 if not(i.startswith('metro_area_') or i.startswith('spot_category_'))]
    # vars_1 = [x[:-15] if x.endswith('_last_4_months\t') else x for x in vars_1]
    vars_1 = sorted(pd.Series(vars_1).unique())
    
    return(vars_, vars_1)

In [33]:
df_res['num_of_removed_vars'] = df_res['models'].map(lambda x: compare_models_vars(x[0], x[1]))\
.map(lambda x: len([i for i in x[0] if i not in x[1]]))

In [34]:
df_res[(df_res['model_number']==40) & (df_res['category'].isin(['low_end_500']))]\
[['model_number', 'category', 'model_name', 'num_of_removed_vars', \
  'variable_name', 'spots_num', 'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

,model_number,category,model_name,num_of_removed_vars,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,40,low_end_500,properly_used_online_orders_inquiries,73,had_properly_used_online_orders_inquiries_last...,219,0.181145,0.189404,0.119482,0.013914


In [35]:
def get_variance(model, variable):
    var_mat = model.variance_matrix_
    var_mat.columns = var_mat.index
    if variable in var_mat.index:
        return(var_mat[var_mat.index==variable][variable].iloc[0])
    else:
        return(pd.NA)

In [36]:
df_res[(df_res['model_number']==40)]\
[['model_number', 'category', 'model_name', 'num_of_removed_vars', \
  'variable_name', 'spots_num', 'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

,model_number,category,model_name,num_of_removed_vars,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,40,high_end_500,properly_used_online_orders_inquiries,84,had_properly_used_online_orders_inquiries_last...,2353,0.993438,0.980105,NaN,NaN
0,40,low_end_500,properly_used_online_orders_inquiries,73,had_properly_used_online_orders_inquiries_last...,219,0.181145,0.189404,0.119482,0.013914
0,40,high_end_700,properly_used_online_orders_inquiries,82,had_properly_used_online_orders_inquiries_last...,2246,0.978642,0.936765,NaN,NaN
0,40,low_end_700,properly_used_online_orders_inquiries,80,had_properly_used_online_orders_inquiries_last...,326,0.539847,0.451112,NaN,NaN
0,40,high_end_1000,properly_used_online_orders_inquiries,91,had_properly_used_online_orders_inquiries_last...,2023,1.096789,0.749796,NaN,NaN
0,40,low_end_1000,properly_used_online_orders_inquiries,78,had_properly_used_online_orders_inquiries_last...,549,0.717318,0.571293,NaN,NaN
0,40,high_end_1200,properly_used_online_orders_inquiries,87,had_properly_used_online_orders_inquiries_last...,1874,1.164636,0.617500,NaN,NaN
0,40,low_end_1200,properly_used_online_orders_inquiries,76,had_properly_used_online_orders_inquiries_last...,698,0.659408,0.399738,NaN,NaN
0,40,high_end_1500,properly_used_online_orders_inquiries,92,had_properly_used_online_orders_inquiries_last...,1665,1.146658,0.672926,NaN,NaN
0,40,low_end_1500,properly_used_online_orders_inquiries,80,had_properly_used_online_orders_inquiries_last...,907,0.872238,0.754278,NaN,NaN


In [37]:
df_res['var_variance_mod0'] = df_res[['variable_name', 'models']].apply(lambda x: get_variance(x['models'][0], 
                                                                 x['variable_name']), axis=1)

df_res['var_variance_mod1'] = df_res[['variable_name', 'models']].apply(lambda x: get_variance(x['models'][1], 
                                                                 x['variable_name']), axis=1)

In [38]:
df_res[(df_res['model_number']==40)]\
[['model_number', 'category', 'model_name', 'num_of_removed_vars', \
  'variable_name', 'spots_num', 'exp(coef)', 'p', 'var_variance_mod0', 
  'exp(coef).2', 'p.2', 'var_variance_mod1']]

,model_number,category,model_name,num_of_removed_vars,variable_name,spots_num,exp(coef),p,var_variance_mod0,exp(coef).2,p.2,var_variance_mod1
0,40,high_end_500,properly_used_online_orders_inquiries,84,had_properly_used_online_orders_inquiries_last...,2353,0.993438,0.980105,0.069691,NaN,NaN,<NA>
0,40,low_end_500,properly_used_online_orders_inquiries,73,had_properly_used_online_orders_inquiries_last...,219,0.181145,0.189404,1.694785,0.119482,0.013914,0.746214
0,40,high_end_700,properly_used_online_orders_inquiries,82,had_properly_used_online_orders_inquiries_last...,2246,0.978642,0.936765,0.074052,NaN,NaN,<NA>
0,40,low_end_700,properly_used_online_orders_inquiries,80,had_properly_used_online_orders_inquiries_last...,326,0.539847,0.451112,0.669243,NaN,NaN,<NA>
0,40,high_end_1000,properly_used_online_orders_inquiries,91,had_properly_used_online_orders_inquiries_last...,2023,1.096789,0.749796,0.083924,NaN,NaN,<NA>
0,40,low_end_1000,properly_used_online_orders_inquiries,78,had_properly_used_online_orders_inquiries_last...,549,0.717318,0.571293,0.344377,NaN,NaN,<NA>
0,40,high_end_1200,properly_used_online_orders_inquiries,87,had_properly_used_online_orders_inquiries_last...,1874,1.164636,0.617500,0.093139,NaN,NaN,<NA>
0,40,low_end_1200,properly_used_online_orders_inquiries,76,had_properly_used_online_orders_inquiries_last...,698,0.659408,0.399738,0.244530,NaN,NaN,<NA>
0,40,high_end_1500,properly_used_online_orders_inquiries,92,had_properly_used_online_orders_inquiries_last...,1665,1.146658,0.672926,0.105099,NaN,NaN,<NA>
0,40,low_end_1500,properly_used_online_orders_inquiries,80,had_properly_used_online_orders_inquiries_last...,907,0.872238,0.754278,0.190720,NaN,NaN,<NA>


In [39]:
df_res[(df_res['model_number']==38)]\
[['model_number', 'category', 'model_name', 'num_of_removed_vars', \
  'variable_name', 'spots_num', 'exp(coef)', 'p', 'var_variance_mod0', 
  'exp(coef).2', 'p.2', 'var_variance_mod1']]

,model_number,category,model_name,num_of_removed_vars,variable_name,spots_num,exp(coef),p,var_variance_mod0,exp(coef).2,p.2,var_variance_mod1
0,38,high_end_500,properly_used_private_parties_inquiries,85,had_properly_used_private_parties_inquiries_la...,2353,0.817289,0.257641,0.031769,NaN,NaN,<NA>
0,38,low_end_500,properly_used_private_parties_inquiries,73,had_properly_used_private_parties_inquiries_la...,219,0.786206,0.860930,1.885111,NaN,NaN,<NA>
0,38,high_end_700,properly_used_private_parties_inquiries,82,had_properly_used_private_parties_inquiries_la...,2246,0.840770,0.339132,0.032921,NaN,NaN,<NA>
0,38,low_end_700,properly_used_private_parties_inquiries,79,had_properly_used_private_parties_inquiries_la...,326,0.373682,0.225528,0.659665,0.265125,0.039990,0.417797
0,38,high_end_1000,properly_used_private_parties_inquiries,90,had_properly_used_private_parties_inquiries_la...,2023,0.841144,0.373546,0.037794,NaN,NaN,<NA>
0,38,low_end_1000,properly_used_private_parties_inquiries,78,had_properly_used_private_parties_inquiries_la...,549,0.572413,0.234122,0.219862,NaN,NaN,<NA>
0,38,high_end_1200,properly_used_private_parties_inquiries,89,had_properly_used_private_parties_inquiries_la...,1874,0.791010,0.253470,0.042148,NaN,NaN,<NA>
0,38,low_end_1200,properly_used_private_parties_inquiries,77,had_properly_used_private_parties_inquiries_la...,698,0.592319,0.153483,0.134626,NaN,NaN,<NA>
0,38,high_end_1500,properly_used_private_parties_inquiries,95,had_properly_used_private_parties_inquiries_la...,1665,0.758101,0.199510,0.046596,NaN,NaN,<NA>
0,38,low_end_1500,properly_used_private_parties_inquiries,81,had_properly_used_private_parties_inquiries_la...,907,0.657062,0.191861,0.103554,NaN,NaN,<NA>


In [ ]:
df_test = 